# Oliva the Friendly Chatbot

Olivia is a friendly chatbot that is powered using a 3 layer Neural Network that uses Nersterov accelerated Stocastic Gradient Descent to learn. 

She is a baby bot that has the capability to greet you, tell you her name, age, and even her weekend plans. We intend to implement a self-learning feature so that you can have interesting conversations with her!



In [165]:
import os
import json
import nltk
import pickle
import random
import string
import sklearn
import numpy as np
from google.colab import drive
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [182]:
nltk.download('punkt')
nltk.download('wordnet')

In [183]:
drive.mount('/content/drive')

In [168]:
DIR = '/content/drive/MyDrive/projects/Chatbot/'
intents_file = os.path.join(DIR, 'intents.json')

In [184]:
with open(intents_file, 'r') as f:
  intents = json.load(f)

# 1. Data Preprocessing

We will preprocess the data in **2 steps**. We will first tokenize the intents file data to get a list of words. We will then lemmatize the words to their canonical forms.

## i. Tokenization

We start by tokenizing the data to break sentences into words.

In the `intents.json` file each tag contains a list of patterns and responses. We will tokenize these patters and add the words into a word list. We will also create a classes and documents list to add all the intents associated with patterns.

In [185]:
words = []
documents = []
classes = set()

In [186]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize pattern
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)        
        # add documents in the corpus
        documents.append((tokens, intent['tag']))
        # add the pattern tag to classes
        classes.add(intent['tag'])

## ii. Word Lemmatization

We will use a `lemmatizer` to convert words into their lemma form. For example, the words "write", "writing", "written", etc... will all be replaced with "write". We use lemmatization to reduce the number of words in our vocabulary.

In [187]:
lemmatizer = WordNetLemmatizer()

In [188]:
# lemmatize and lower words
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# remove duplicates
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(classes))

# 2. Data Preparation

We will create **Bag of Words** for word patterns and intents, which will be used to train our model.

In [189]:
bag_patterns = []
bag_intents = []

for document in documents:
    patterns, intent = document

    patterns_row = []
    patterns = [lemmatizer.lemmatize(word.lower()) for word in patterns]
    for word in words:
        patterns_row.append(1) if word in patterns else patterns_row.append(0)
    bag_patterns.append(patterns_row)

    intents_row = [0] * len(classes)
    intents_row[classes.index(intent)] = 1
    bag_intents.append(intents_row)

In [190]:
train_X, train_y = sklearn.utils.shuffle(bag_patterns, bag_intents)

# 3. Model Training

We will use a neural network consisting of 3 dense layers with *128*, *64*, and *c* neurons respectively, where c is the number of classes. The dropout layers reduce overfitting.

We will also use the SGD to optimize our model. 

In [191]:
# Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [192]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_X), np.array(train_y), epochs=100, batch_size=5, verbose=1)

Epoch 1/100
4/4 [==============================] - 1s 6ms/step - loss: 1.7434 - accuracy: 0.1000
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 1.6253 - accuracy: 0.1500
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5759 - accuracy: 0.3500
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5493 - accuracy: 0.5000
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 1.5220 - accuracy: 0.3500
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 1.5362 - accuracy: 0.3000
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4213 - accuracy: 0.4000
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 1.3931 - accuracy: 0.5000
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3340 - accuracy: 0.5500
Epoch 10/100
4/4 [==============================] - 0s 5ms/step - loss: 1.3022 - accuracy: 0.5500
Epoch 11/100
4/4 [===========

In [193]:
def bag_of_words(sentence): 
    tokens = nltk.word_tokenize(sentence)
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    bag = [0] * len(words)
    for token in tokens: 
        for i, word in enumerate(words):
            if token == word: 
                bag[i] = 1
    return np.array(bag)

In [194]:
def pred_class(sentence): 
    bag = bag_of_words(sentence)
    result = model.predict(np.array([bag]))[0]
    THRESHOLD = 0.25
    y_pred = [[i, res] for i, res in enumerate(result) if res > THRESHOLD]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(classes[r[0]])
    return return_list

In [195]:
def get_response(tags, intents): 
    tag = tags[0]
    list_of_intents = intents["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [196]:
while True:
    message = input("You: ")
    c = pred_class(message)
    result = get_response(c, intents)
    print("Bot :", result)
    print ()
    if c[0] == "goodbye":
        break

You: Hello
Bot : How are you doing?

You: What is your name?
Bot : I'm Olivia

You: How old are you?
Bot : My birthday is June 4th and I was born in 1998

You: What are you doing on the weekend?
Bot : I am not busy

You: Any plans for the weekend?
Bot : I am available all week

You: When were you born?
Bot : My birthday is June 4th and I was born in 1998

You: Bie
Bot : Speak soon!

